In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Setup

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## Load the data

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

## Data exploration

In [ ]:
train_data.describe()

In [ ]:
train_data.dtypes

In [ ]:
train_data.isnull().any()

In [ ]:
train_data_numeric = train_data.select_dtypes(exclude='object')

In [ ]:
train_data_numeric.describe()

In [ ]:
fig,ax=plt.subplots()

n_bins=5
bins = np.linspace(train_data_numeric['Age'].min(), train_data_numeric['Age'].max(), n_bins)
for Pclass, df in train_data_numeric.groupby(by='Pclass'):
    sns.distplot(df['Age'], norm_hist=True, ax=ax, hist=False, bins=bins, label='%i class' % Pclass)

ax.legend();

In [ ]:
fig,ax=plt.subplots()
class_groups = train_data_numeric.groupby(by='Pclass')
class_groups['Survived'].mean().plot(kind='bar', ax=ax)
ax.set_ylabel('Survival');

In [ ]:
gender_groups = train_data.groupby(by='Sex')
gender_groups['Survived'].mean()

In [ ]:
mask = (train_data['Survived']==1)
df_survivers = train_data.loc[mask].copy()

bins = np.linspace(0, df_survivers['Age'].max(),30)
for gender, df_gender in df_survivers.groupby(by=['Sex']):
    fig,ax=plt.subplots()
    
    for Pclass, df in df_gender.groupby(by=['Pclass']):
        sns.distplot(df['Age'], norm_hist=True, ax=ax, bins=bins, hist=True, kde=False, label='%s' % (Pclass,))

    ax.set_title(gender)
    ax.legend();

## Fit

In [ ]:
y = train_data["Survived"]
features = ["Pclass", "Sex", "SibSp", "Parch", "Age"]
X = pd.get_dummies(train_data[features])

## Handle age=Null

In [ ]:
mask = X['Age'].isnull()
X.loc[mask]

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X['Age'] = imputer.fit_transform(X[['Age']])
test_data['Age'] = imputer.transform(test_data[['Age']])

## RandomForestClassifier

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [ ]:
X.isnull().any()

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

In [ ]:
scores = cross_val_score(estimator=model, X=X, y=y, scoring="accuracy", cv=cv, n_jobs=-1)

In [ ]:
fig,ax = plt.subplots()
ax.hist(scores);
ax.set_xlabel('score')
ax.set_ylabel('occurances')
ax.set_title('Histogram over cross validations');


In [ ]:
from sklearn.model_selection import GridSearchCV

# define the grid
param_grid = { 
    'n_estimators': [200, 500, 1000],
    'max_features': ['auto',],
    'max_depth' : [3,4,5,6],
    'criterion' :['gini', 'entropy']
}



# define the grid search
search = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
search.fit(X, y)


In [ ]:
search.best_params_

In [ ]:
search.best_score_

In [ ]:
scores = cross_val_score(estimator=search.best_estimator_, X=X, y=y, scoring="accuracy", cv=cv, n_jobs=-1)

fig,ax = plt.subplots()
ax.hist(scores);
ax.set_xlabel('score')
ax.set_ylabel('occurances')
ax.set_title('Histogram over cross validations');

In [ ]:
model = search.best_estimator_

In [ ]:
y = train_data["Survived"]

X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")